In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

## Bernstein Vazirani
#### In this tutorial, we will demonstrate the capabilities of qBraid Algorithms Bernstein Vazirani Module
Begin by importing the module from qBraid Algorithms library

In [2]:
import pyqasm
from qbraid_algorithms import bernstein_vazirani as bv

To load a full Bernstein Vazirani algorithm circuit as a PyQASM module, simply pass the secret string to be encoded in the oracle to the `load_program()` method.

In [3]:
module = bv.load_program('1011')

We can perform all standard [PyQASM](https://docs.qbraid.com/pyqasm/user-guide/overview) operations on the module, such as unrolling.

In [4]:
module.unroll()

ERROR:pyqasm: Error at line 8, column 0 in QASM file

 >>>>>> bernvaz(q, ancilla)



ValidationError: Undefined subroutine 'bernvaz' was called

Below, we display the unrolled circuit, which includes preparing the input and ancilla qubits, applying the '1011' oracle, applying Hadamard gates after the oracle, then measuring the results.

In [5]:
module_str = pyqasm.dumps(module)
print(module_str)

OPENQASM 3.0;
include "bernvaz_subroutine.qasm";
qubit[4] q;
qubit[1] ancilla;
bit[4] b;
bernvaz(q, ancilla);
b = measure q;



## Using B-V in your own OpenQASM3 program
#### qBraid algorithms makes it easy to incorporate either the full Bernstein Vazirani algorithm - or just the encoded oracle - into your own OpenQASM3 circuit.
To use a secret string-encoded oracle in your circuit, first generate the oracle submodule using the `generate_oracle` method, which takes a secret string. The method will create a QASM3 file containing your oracle as a subroutine within your current working directory.

In [6]:
bv.generate_oracle('111')

Oracle 'oracle' has been added to /Users/lukeandreesen/qbraid_algos/examples/oracle.qasm


Below, we can see the custom oracle subroutine that you now have access to. To use this in your own circuit, simply add `include "oracle.qasm";` to your OpenQASM file, and call the `oracle` subroutine by passing an appropriately sized register of qubits and an ancilla qubit.

In [7]:
%cat oracle.qasm

OPENQASM 3.0;
include "stdgates.inc";

def oracle(qubit[3] q, qubit[1] ancilla) {
    int[32] s = 7;
    int[16] n = 3;
    for int i in [0:n - 1] {
        if ((s >> i) & 1) {
            cx q[i], ancilla[0];
        }
    }
}


Similarly, you can generate an entire Bernstein Vazirani circuit as a submodule using the `generate_subroutine` method, again passing your desired secret string.

In [8]:
subroutine = bv.generate_subroutine('011')

Subroutine 'bernvaz' has been added to /Users/lukeandreesen/qbraid_algos/examples/bernvaz.qasm


To use the subroutine in your own circuit, add `include "bernvaz.qasm";` to your OpenQASM file, and call the `bernvaz` method by passing an appropriately sized register of qubits, as well as an ancilla qubit.

In [9]:
%cat bernvaz.qasm

OPENQASM 3.0;
include "stdgates.inc";

def bernvaz(qubit[3] q, qubit[1] ancilla) {
    int[32] s = 6;
    int[16] n = 3;
    for int i in [0:n - 1] {
        h q[i];
    }
    x ancilla[0];
    h ancilla[0];
    // Note: Nested subroutine calls not yet supported by QASM, so manually insert
    for int i in [0:n - 1] {
        if ((s >> i) & 1) {
            cx q[i], ancilla[0];
        }
    }
    for int i in [0:n - 1] {
        h q[i];
    }

}
